# Keras Overview

An overview of using Keras 3 for building an autoencoder and using it for anomaly detection.

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('jax','jax'),
    ('keras', 'keras', '3.6.0'),
    ('pydot', 'pydot')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Graphviz Install

Plotting the [model structure with Keras](https://keras.io/api/utils/model_plotting_utils/) uses [Graphviz](https://graphviz.org/download/).

This code check for Graphviz and if missing installs it.

In [35]:
check = !dot -V
if check[0].startswith('dot'):
    print(f'Graphviz installed with version: {check[0]}')
else:
    print('Installing Graphviz...')
    install = !sudo apt-get install graphviz --assume-yes
    print('Completed')

Graphviz installed with version: dot - graphviz version 2.43.0 (0)


### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'frameworks-keras'
EXPERIMENT = 'overview'

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

# make this the BigQuery Project / Dataset / Table prefix to store results
BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-', '_')
BQ_TABLE = EXPERIMENT
BQ_REGION = REGION[0:2] # use a multi region

Packages

In [8]:
import os#, json, time, glob

#import numpy as np

# import keras, set backend prior to first import
os.environ['KERAS_BACKEND'] = 'jax'
import keras

# Vertex AI
from google.cloud import aiplatform

# BigQuery
from google.cloud import bigquery

In [9]:
aiplatform.__version__

'1.71.0'

In [10]:
keras.__version__

'3.6.0'

Clients

In [11]:
# vertex ai clients
aiplatform.init(project = PROJECT_ID, location = REGION)

# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

---
## Review Source Data

This is a BigQuery public table of 284,807 credit card transactions classified as fradulant or normal in the column `Class`.
- The data can be researched further at this [Kaggle link](https://www.kaggle.com/mlg-ulb/creditcardfraud).
- Read mode about BigQuery public datasets [here](https://cloud.google.com/bigquery/public-data)

In order protect confidentiality, the original features have been transformed using [principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) into 28 features named `V1, V2, ... V28` (float).  Two descriptive features are provided without transformation by PCA:
- `Time` (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- `Amount` (float) is the value of the transaction
 

### Review BigQuery table:

In [12]:
source_data = bq.query(f'SELECT * FROM `{BQ_SOURCE}` LIMIT 5').to_dataframe()
source_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8748.0,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.0,0
1,27074.0,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.0,0
2,28292.0,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.0,0
3,28488.0,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.0,0
4,31392.0,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.0,0


In [13]:
source_data.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       Int64
dtype: object

---
## Prepare Data Source

The data preparation includes adding splits for machine learning with a column named `splits` with 80% for training (`TRAIN`), 10% for validation (`VALIDATE`) and 10% for testing (`TEST`).  Additionally, a unique identifier was added to each transaction, `transaction_id`. 

### Create/Recall Dataset

In [14]:
dataset = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
dataset.location = BQ_REGION
bq_dataset = bq.create_dataset(dataset, exists_ok = True)

### Create/Recall Table With Preparation For ML

Copy the data from the source while adding columns:
- `transaction_id` as a unique identify for the row
    - Use the `GENERATE_UUID()` function
- `splits` column to randomly assign rows to 'TRAIN", "VALIDATE" and "TEST" groups
    - stratified sampling within the levels of `class` by first assigning row numbers within the levels of `class` then using the with a CASE statment to assign the `splits` level.

In [15]:
job = bq.query(f"""
#CREATE OR REPLACE TABLE
CREATE TABLE IF NOT EXISTS 
    `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS
WITH
    add_id AS (
        SELECT *,
            GENERATE_UUID() transaction_id,
            ROW_NUMBER() OVER (PARTITION BY class ORDER BY RAND()) as rn
            FROM `{BQ_SOURCE}`
    )
SELECT * EXCEPT(rn),
    CASE 
        WHEN rn <= 0.8 * COUNT(*) OVER (PARTITION BY class) THEN 'TRAIN'
        WHEN rn <= 0.9 * COUNT(*) OVER (PARTITION BY class) THEN 'VALIDATE'
        ELSE 'TEST'
    END AS splits
FROM add_id
""")
job.result()
(job.ended-job.started).total_seconds()

0.288

In [16]:
raw_sample = bq.query(f'SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` LIMIT 5').to_dataframe()
raw_sample

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,43330.0,-1.510308,0.780684,2.085747,3.123133,0.589557,1.301681,-1.300991,-2.345333,0.148722,...,-0.341324,-0.370869,-0.201199,0.423964,0.210998,-0.058616,0.0,0,34a1f550-443c-45f0-858d-94cf0baebfe8,TEST
1,141955.0,-1.585532,1.240411,-0.558349,-0.841657,0.397502,-1.532723,0.388818,0.575896,-0.278370,...,-0.249493,0.001462,-0.045791,-0.110667,0.305249,0.015271,0.0,0,a2dfc3ff-aa20-4742-bb1c-c6f67440e34a,TEST
2,58813.0,-2.524342,-0.313784,2.304085,3.043016,5.495686,-2.600347,-3.779679,-1.042154,-1.014503,...,-6.185491,0.440080,-1.409641,-0.209465,-0.111065,0.116788,0.0,0,d7910b3a-fc5e-417c-8b61-5516cc4dc981,TEST
3,139716.0,-2.006121,2.048709,-0.963971,-1.046216,-0.118273,-1.400030,0.401296,0.728337,0.095944,...,0.059168,-0.113542,0.066162,-0.143905,-0.218363,-0.091904,0.0,0,4de5ecd4-7c82-40dd-a52a-4b5e8b1e8ba3,TEST
4,59183.0,1.175167,-0.087900,0.278759,-0.279214,-0.166236,0.058772,-0.232681,0.192241,-0.014614,...,0.165047,-0.253526,-0.016523,0.890504,-0.059400,-0.013996,0.0,0,ec122bde-5780-4223-9e35-7f26de453512,TEST


### Review the number of records for each level of `Class` for each of the data splits:

In [17]:
bq.query(f"""
SELECT splits, class,
    count(*) as count,
    ROUND(count(*) * 100.0 / SUM(count(*)) OVER (PARTITION BY class), 2) AS percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
GROUP BY splits, class
""").to_dataframe()

,splits,class,count,percentage
0,TEST,1,50,10.16
1,TRAIN,1,393,79.88
2,VALIDATE,1,49,9.96
3,TEST,0,28432,10.00
4,TRAIN,0,227452,80.00
5,VALIDATE,0,28431,10.00


## Training - Local, Simple

In [86]:
train_ds.dtypes

Time              float64
V1                float64
V2                float64
V3                float64
V4                float64
V5                float64
V6                float64
V7                float64
V8                float64
V9                float64
V10               float64
V11               float64
V12               float64
V13               float64
V14               float64
V15               float64
V16               float64
V17               float64
V18               float64
V19               float64
V20               float64
V21               float64
V22               float64
V23               float64
V24               float64
V25               float64
V26               float64
V27               float64
V28               float64
Amount            float64
Class               Int64
transaction_id     object
dtype: object

In [97]:
train_ds = bq.query(f"SELECT * EXCEPT(splits) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'TRAIN'").to_dataframe()
test_ds = bq.query(f"SELECT * EXCEPT(splits) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'TEST'").to_dataframe()
validate_ds = bq.query(f"SELECT * EXCEPT(splits) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'VALIDATE'").to_dataframe()

In [98]:
# Define column categories
var_class = ['Class']
var_omit = ['transaction_id']
var_numeric = [x for x in train_ds.columns.tolist() if x not in var_class + var_omit]

In [99]:
# Feature Layers
numeric_feature_inputs = [keras.Input(shape = (1,), dtype = 'float32', name = feature) for feature in var_numeric]
class_feature_inputs = [keras.Input(shape = (1,), dtype = 'int32', name = feature) for feature in var_class]

In [100]:
# feature grouping layers
numeric_concat = keras.layers.Concatenate(name = 'numeric_features')(numeric_feature_inputs)
class_concat = keras.layers.Concatenate(name = 'class_features')(class_feature_inputs)

In [101]:
# Create a single Normalization layer for all numeric features
normalizer = keras.layers.Normalization(axis=-1, name='normalize_numeric')
normalizer.adapt(train_ds[var_numeric].to_numpy())  # Adapt on all numeric columns

# Apply normalization to the concatenated numeric features
normalized_numeric = normalizer(numeric_concat)

In [102]:
# --- One-hot Encode Class Features --
onehot_encoded_classes = []
for i, col in enumerate(var_class):
    class_levels = train_ds[col].unique()
    if train_ds[col].dtype == 'object':
        indexer = keras.layers.StringLookup(vocabulary=class_levels, output_mode='one_hot', name=f'onehot_{col}')
    else:
        indexer = keras.layers.IntegerLookup(vocabulary=class_levels, output_mode='one_hot', name=f'onehot_{col}')
    onehot_encoded_classes.append(indexer(class_feature_inputs[i]))
    
# Concatenate the one-hot encoded class features
encoded_class_concat = keras.layers.Concatenate(name='concat_onehot')(onehot_encoded_classes)

In [103]:
# --- Concatenate Normalized Numeric and Encoded Class Features ---
all_features_concat = keras.layers.Concatenate(name='all_features')([normalized_numeric, encoded_class_concat])

In [104]:
# --- Encoder and Decoder Layers ---
encoded = keras.layers.Dense(24, activation='relu', name='enc_dense1')(all_features_concat)
encoded = keras.layers.Dropout(0.2, name='enc_dropout1')(encoded)
encoded = keras.layers.Dense(16, activation='relu', name='enc_dense2')(encoded)
encoded = keras.layers.Dropout(0.2, name='enc_dropout2')(encoded)
encoded = keras.layers.Dense(8, activation='relu', name='enc_dense3')(encoded)

decoded = keras.layers.Dense(16, activation='relu', name='dec_dense1')(encoded)
decoded = keras.layers.Dropout(0.2, name='dec_dropout1')(decoded)
decoded = keras.layers.Dense(24, activation='relu', name='dec_dense2')(decoded)
decoded = keras.layers.Dropout(0.2, name='dec_dropout2')(decoded)
decoded = keras.layers.Dense(33, activation='linear', name='dec_dense3')(decoded)

In [105]:
# Split the decoded output into numeric and class features
decoded_numeric = keras.layers.Lambda(lambda x: x[:, :len(var_numeric)], name='decoded_numeric')(decoded)
decoded_class = keras.layers.Lambda(lambda x: x[:, len(var_numeric):], name='decoded_class')(decoded)

# Denormalize the numeric features
denormalized_numeric = normalizer(decoded_numeric)  # Use the same 'normalizer'

# Split the denormalized numeric output and decoded class output into individual columns
decoded_outputs = []
for i, col in enumerate(var_numeric):
    decoded_outputs.append(keras.layers.Lambda(lambda x: x[:, i], name=f'decoded_{col}')(denormalized_numeric))
for i, col in enumerate(var_class):
    decoded_outputs.append(keras.layers.Lambda(lambda x: x[:, i], name=f'decoded_{col}')(decoded_class))

# --- Create and Compile the Model ---
all_inputs = numeric_feature_inputs + class_feature_inputs  # Combine input lists
autoencoder = keras.Model(inputs = all_inputs, outputs = [encoded, decoded_outputs], name = 'autoencoder')
autoencoder.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.MeanAbsoluteError(),
    metrics = [
        keras.metrics.MeanAbsoluteError(name = 'mae'),
        keras.metrics.RootMeanSquaredError(name = 'rmse')
    ]
)

In [106]:
autoencoder.summary()

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Time (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V1 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V2 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V3 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V4 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V5 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V6 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V7 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V8 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V9 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V10 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V11 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V12 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V13 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V14 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V15 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V16 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V17 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V18 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V19 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V20 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V21 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V22 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V23 (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ V24 (InputLayer)    │ (None, 1)         │          0 │ -               

 Total params: 2,790 (10.90 KB)

 Trainable params: 2,729 (10.66 KB)

 Non-trainable params: 61 (244.00 B)

In [107]:
keras.utils.plot_model(autoencoder, show_shapes = True, show_layer_names = True, rankdir = 'LR')

In [111]:
sample_input = [
    train_ds[col].astype('float32').to_numpy()[:10] for col in var_numeric
] + [
    train_ds[col].astype('int32').to_numpy()[:10] for col in var_class
]

# Call the model on the sample input
autoencoder(sample_input)

[Array([[0.11687022, 0.06159715, 0.        , 0.        , 0.24235909,
         0.        , 0.1482341 , 0.        ],
        [0.665264  , 0.        , 0.72711647, 0.70449775, 0.        ,
         0.        , 0.08463688, 0.        ],
        [0.3761079 , 0.        , 0.29519236, 0.10426056, 0.14757353,
         0.        , 0.        , 0.10328239],
        [0.04264659, 0.        , 0.2516866 , 0.        , 0.11977978,
         0.02270181, 0.11409742, 0.        ],
        [0.27067858, 0.        , 0.        , 0.        , 0.8975395 ,
         0.        , 0.07666299, 0.        ],
        [0.6932825 , 0.16956544, 0.        , 0.        , 0.6347864 ,
         0.        , 0.37838113, 0.7119424 ],
        [0.3663717 , 0.        , 0.6751447 , 0.        , 0.8348045 ,
         0.        , 0.8013902 , 0.        ],
        [0.73684955, 0.22046721, 0.        , 0.        , 0.63033354,
         0.        , 0.01146031, 0.12135813],
        [0.43523368, 0.        , 0.3132237 , 0.        , 0.75983775,
         0.

In [109]:
history = autoencoder.fit(
    [train_ds[col].astype('float32') for col in var_numeric]+[train_ds[col].astype('int32') for col in var_class],
    [train_ds[col].astype('float32') for col in var_numeric]+[train_ds[col].astype('int32') for col in var_class],
    epochs = 10,
    batch_size = 100,
    validation_data = (
        [validate_ds[col].astype('float32') for col in var_numeric]+[validate_ds[col].astype('int32') for col in var_class],
        [validate_ds[col].astype('float32') for col in var_numeric]+[validate_ds[col].astype('int32') for col in var_class]
    )
)

RuntimeError: Unable to automatically build the model. Please build it yourself before calling fit/evaluate/predict. A model is 'built' when its variables have been created and its `self.built` attribute is True. Usually, calling the model on a batch of data is the right way to build it.
Exception encountered:
'Exception encountered when calling Functional.call().

[1m'str' object has no attribute '_error_repr'[0m

Arguments received by Functional.call():
  • inputs=['jnp.ndarray(shape=(100, 1), dtype=float32)', 'jnp.ndarray(shape=(100, 1), dtype=float32)', 'jnp.ndarray(shape=(100, 1), dtype=float32)', 'jnp.ndarray(shape=(100, 1), dtype=float32)', 'jnp.ndarray(shape=(100, 1), dtype=float32)', 'jnp.ndarray(shape=(100, 1), dtype=float32)', 'jnp.ndarray(shape=(100, 1), dtype=float32)', 'jnp.ndarray(shape=(100, 1), dtype=float32)', 'jnp.ndarray(shape=(100, 1), dtype=float32)', 'jnp.ndarray(shape=(100, 1), dtype=float32)', 'jnp.ndarray(shape=(100, 1), dtype=float32)', 'jnp.ndarray(shape=(

In [84]:
[train_ds[col] for col in var_class]

[0         0
 1         0
 2         0
 3         0
 4         0
          ..
 227840    0
 227841    0
 227842    0
 227843    0
 227844    0
 Name: Class, Length: 227845, dtype: Int64]